In [5]:
w = torch.tensor([1, 2])
w.reshape((-1,1))

tensor([[1],
        [2]])

In [ ]:
!pip install d2l==1.0.0-beta0

Học máy là tất cả về trích xuất thông tin từ dữ liệu. Vì vậy, bạn có thể tự hỏi, chúng ta có thể học được gì từ dữ liệu tổng hợp? Mặc dù về bản chất, chúng ta có thể không quan tâm đến các mẫu mà chính chúng ta đã đưa vào mô hình tạo dữ liệu nhân tạo, nhưng các bộ dữ liệu đó vẫn hữu ích cho các mục đích mô phạm, giúp chúng ta đánh giá các thuộc tính của thuật toán học tập và xác nhận rằng các triển khai của chúng ta hoạt động như mong đợi. Ví dụ: nếu chúng tôi tạo dữ liệu có các tham số chính xác được biết trước , thì chúng tôi có thể xác minh rằng mô hình của chúng tôi trên thực tế có thể khôi phục chúng.

In [3]:
%matplotlib inline
import random
import torch
from d2l import torch as d2l

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


# 3.3.1. Tạo tập dữ liệu

Với ví dụ này, ta sẽ làm việc với chiều thấp cho ngắn gọn. Code dưới dây sẽ tạo ra 1000 mẫu với 2 chiều được rút ra từ 1 phân phối chuẩn thông thường. Ma trận thiết kế kết quả X thuộc $R^{1000x2}$. Chúng ta sẽ tạo từng nhãn bằng cách áp dụng hàm tuyến tính thực tế cơ bản, thêm nhiễu:
**$$\mathbf{y}= \mathbf{X} \mathbf{w} + b + \mathbf\epsilon.$$

Để thuận tiện, ta giả định rằng nhiễu được rút ra từ một phân phối bình thường với mean = 0 và std = 0.01. Lưu ý rằng, với thiết kế hướng đối tượng, ta thêm mã vào phương thức `__init__` của lớp con `d2l.DataModule`. 

In [8]:
class SyntheticRegressionData(d2l.DataModule):
  """Tạo data từ phương trình hồi quy tuyến tính"""
  def __init__(self, w, b, noise = 0.01, num_train = 1000, num_val = 1000,
               batch_size = 32):
    super().__init__()
    self.save_hyperparameters()
    n = num_train + num_val
    self.X = torch.randn(n, len(w))
    noise = torch.randn(n, 1) * noise
    self.y = torch.matmul(self.X, w.reshape((-1, 1))) + b + noise

Ta thử với $w = [2, -3.4]^T$ và $b = 4.2$

In [11]:
data = SyntheticRegressionData(w = torch.tensor([2, -3.4]), b = 4.2)

Xem thử chỉ mục đầu tiên

In [12]:
print('features:', data.X[0],'\nlabel:', data.y[0])

features: tensor([ 1.4874, -1.3539]) 
label: tensor([11.7824])


# 3.3.2. Đọc tập dữ liệu

Đào tạo các mô hình máy học thường yêu cầu nhiều lượt chuyển qua một tập dữ liệu, lấy một nhóm nhỏ các ví dụ tại một thời điểm. Dữ liệu này sau đó được sử dụng để cập nhật mô hình. Để minh họa cách thức hoạt động của nó, chúng tôi triển khai get_dataloaderphương thức, đăng ký nó trong SyntheticRegressionDatalớp thông qua add_to_class(được giới thiệu trong Phần 3.2.1 ).

Nó lấy một kích thước lô, một ma trận các tính năng và một vectơ nhãn, đồng thời tạo ra các lô nhỏ có kích thước batch_size. Như vậy, mỗi minibatch bao gồm một bộ tính năng và nhãn. Lưu ý rằng chúng tôi cần lưu ý xem chúng tôi đang ở chế độ đào tạo hay xác thực: trước đây, chúng tôi sẽ muốn đọc dữ liệu theo thứ tự ngẫu nhiên, trong khi đối với chế độ sau, việc có thể đọc dữ liệu theo thứ tự được xác định trước có thể quan trọng cho mục đích gỡ lỗi.


In [13]:
@d2l.add_to_class(SyntheticRegressionData)
def get_dataloader(self, train):
  if train:
    indices = list(range(0, self.num_train))
    random.shuffle(indices)
  else:
    indices = list(range(self.num_train, self.num_train+self.num_val))
  for i in range(0, len(indices), self.batch_size):
    batch_indices = torch.tensor(indices[i: i + self.batch_size])
    yield self.X[batch_indices], self.y[batch_indices]

Để xây dựng một số trực giác, hãy kiểm tra minibatch dữ liệu đầu tiên. Mỗi minibatch của các tính năng cung cấp cho chúng tôi cả kích thước và kích thước của các tính năng đầu vào. Tương tự như vậy, lô nhãn nhỏ của chúng tôi sẽ có hình dạng phù hợp được cung cấp bởi batch_size.

In [14]:
X, y = next(iter(data.train_dataloader()))
print('X shape:', X.shape, '\ny shape:', y.shape)

X shape: torch.Size([32, 2]) 
y shape: torch.Size([32, 1])


Trong suốt quá trình lặp lại, chúng tôi thu được các lô nhỏ riêng biệt cho đến khi toàn bộ tập dữ liệu đã cạn kiệt (hãy thử điều này). Mặc dù phép lặp được triển khai ở trên là tốt cho các mục đích mô phạm, nhưng nó không hiệu quả theo cách có thể khiến chúng ta gặp rắc rối với các vấn đề thực tế. Ví dụ: nó yêu cầu chúng tôi tải tất cả dữ liệu trong bộ nhớ và chúng tôi thực hiện nhiều truy cập bộ nhớ ngẫu nhiên. Các trình vòng lặp tích hợp được triển khai trong khung học sâu hiệu quả hơn đáng kể và chúng có thể xử lý các nguồn như dữ liệu được lưu trữ trong tệp, dữ liệu nhận được qua luồng và dữ liệu được tạo hoặc xử lý nhanh chóng. Tiếp theo, hãy thử thực hiện cùng một phương thức bằng cách sử dụng các trình vòng lặp tích hợp sẵn.

# 3.3.3. Triển khai xúc tích Data Loader

Thay vì viết iterator của riêng mình, chúng ta có thể gọi API hiện có trong một khung để tải dữ liệu. Như trước đây, chúng ta cần tập dữ liệu có các features X và labels y. Ngoài ra, chúng ta đã thiết lập batch_size tích hợp sẵn vào data loader và để nó xử lý việc xáo trộn các ví dụ một cách hiệu quả.

In [ ]:
@d2l.add_to_class(d2l.DataModule)
def get_tensorloader(self, tensors, train, indices = slice(0, None)):
  tensors = tuple(a[indices] for a in tensors)
  dataset = torch.utils.data.TensorDataset(*tensors)
  return torch.utils.data.DataLoader(dataset, self.batch_size,
                                      shuffle=train)
@d2l.add_to_class(SyntheticRegressionData)
def get_dataloader(self, train):
  i = slice(0, self.num_train) if train else slice(self.num_train, None)
  return self.get_tensorloader((self.X, self.y), train, i)

Cách này hoạt động tương tự cách trên nhưng hiệu quả hơn và có thêm 1 số chức năng.

In [17]:
X, y = next(iter(data.train_dataloader()))
print('X shape:', X.shape, '\ny shape:', y.shape)


X shape: torch.Size([32, 2]) 
y shape: torch.Size([32, 1])


In [18]:
len(data.train_dataloader())

TypeError: ignored